# This is a first attempt 

# Prime cycles of length N with K symbols.

Starts with an array of K symbols, and then recursively builts up the prime cycles. Basically just tacks on each symbol to every prime cycle of length N-1 and then checks
for repeats. I think that that works if permutations aren't used to build up the cycles at any point.

In [1]:
import itertools
from functools import lru_cache
import numpy as np
from scipy.special import comb
import time

In [2]:
def n_prime_cycles(n, k):
    if n == 1:
        return k
    elif n == 2:
        return int(comb(k, 2))
    else:
        return int((1./n) * (k ** n - np.sum([n_prime_cycles(j, k) * j for j in range(1, n//2 + 1) if n % j == 0])))

# Number of primes up to cyclic rotation

$q(n+1, k) = \frac{1}{n}(k^n - q(n, k) * k)$

$q(n,k)$ is the number of primes of length $n$ with $k$ symbols 

The way primes are found: we start with a set of cycles and their doubles e.g. '100' and '100100'. All original cycles are compared with these doubled cycles; specifically we count whether the substring occurs in the doubled string. We collect all cycles which are contained in each double, e.g. 100, 010, 001 are in 100100. The originals are sorted, and the zeroth element is chosen to represent the group orbit; '001' in this case. 

This is done for all cycles, so '001' would actually be added three times via 100100, 010010 and 001001, therefore we must
take only the unique results. 


In [3]:
n = 12
k = 3
states = [str(x) for x in range(k)]

In [4]:
@lru_cache
def prime_cycles(n, k):
    """
    Returns prime cycles of k-ary symbolic dynamics; only quotients cyclic rotations and repeats  
    
    n : int 
        length of cycles
    k : int
        k-ary symbolic dynamics
    cumulative : bool
        If True then returns all prime cycles of length 1 up to length n
    
    """
    
    
    if n == 1:
        return  [str(x) for x in range(k)]
    if n == 2:
        return [''.join(x) for x in list(itertools.combinations(prime_cycles(1, k), 2))] + prime_cycles(1, k)
    else:

        shorter_primes = prime_cycles(n-1, k)
        all_states = np.concatenate([coord.ravel().reshape(-1, 1) 
                                     for coord in np.meshgrid(*(states for i in range(n)))], axis=1)
        candidates = np.sum(np.array(all_states, dtype=object), axis=1)
        doubles = np.array([2*x for x in candidates])
        primes = []
        for ci in candidates:
            temp = np.char.find(doubles, ci)
            where_matches = np.where(temp!=-1)
            matches = candidates[where_matches]
            primes.append(np.sort(matches)[0])
        primes = np.unique(primes)
        for each_shorter_prime in shorter_primes:
            if (not (n % len(each_shorter_prime))) or len(each_shorter_prime) == 1:
                counts = np.char.count(primes, each_shorter_prime)
                lengths = counts * len(each_shorter_prime)
                primes = primes[np.where(lengths != n)[0]]

        print(f'For {n}-cycles: (N initial states, N results, N primes)={len(candidates), len(primes),  n_prime_cycles(n, k)}; if correct then '
              f'N results == N primes')
        return np.concatenate((primes, shorter_primes))

In [5]:
n, k = 9, 3
t0 = time.time_ns()/10**9
cycles = prime_cycles(n, k)
t1 = time.time_ns()/10**9
print(f'It tooks {t1-t0} seconds to compute all prime cycles up to length {n} with {k} symbols')

For 3-cycles: (N initial states, N results, N primes)=(27, 8, 8); if correct then N results == N primes
For 4-cycles: (N initial states, N results, N primes)=(81, 18, 18); if correct then N results == N primes
For 5-cycles: (N initial states, N results, N primes)=(243, 48, 48); if correct then N results == N primes
For 6-cycles: (N initial states, N results, N primes)=(729, 116, 116); if correct then N results == N primes
For 7-cycles: (N initial states, N results, N primes)=(2187, 312, 312); if correct then N results == N primes
For 8-cycles: (N initial states, N results, N primes)=(6561, 810, 810); if correct then N results == N primes
For 9-cycles: (N initial states, N results, N primes)=(19683, 2184, 2184); if correct then N results == N primes
It tooks 152.65200066566467 seconds to compute all prime cycles up to length 9 with 3 symbols
